1. Number of updated projects of both sheets since and before (by donor agency and by SWG)(The number of updated projects refers to both the online Excel sheet and Google form)
2. Number of updated projects on the online Excel sheet only (whose online form is not updated)( by donor agency and by SWG)
3. Number of new projects and list registered ( by donor agency and by SWG)
4. Number of not updated projects ( by donor agency and by SWG)
5. Can we get an Excel sheet under each SWG updated and not updated project info of both online and Excel sheet, updated fields colored?
6. Can we get all SWG info in one file in the above similar request?

On the AMP system or new system or online form: create a links for the user like existing since, existing before, new project, select donor org (select yours in the case of multi), etc. when all infor filled filter out specific project and take the data encoder to relevant page, meaning page with filtered out projects.

In [93]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
from openpyxl.utils import get_column_letter

In [94]:
# get_ids_and_titles
def get_ids_and_titles(original, latest):
    # Retrieve values from specific columns
    original_titles = original.iloc[7:, 2].values
    latest_titles = latest.iloc[7:, 2].values
    original_ids = original.iloc[7:, 7].values
    latest_ids = latest.iloc[7:, 7].values

    return original_titles, latest_titles, original_ids, latest_ids   

In [95]:
def compare_stat(original, latest):
    # 3. Create dictionaries for categorizing projects
    new_projs = {}
    original_not_updated_projs = {}
    original_updated_projs = {}
    deleted_projs = {}

    _, latest_titles, original_ids, latest_ids = get_ids_and_titles(original, latest)

    # get all ids which are in original but not in latest
    for i, original_id in enumerate(original_ids):
        if original_id not in latest_ids:
            deleted_projs[original_id] = original.iloc[i + 7]

    # 4. Iterate over latest IDs for comparison
    for i, latest_id in enumerate(latest_ids): 
        # declare empty index and row variables   

        if latest_id in original_ids:
            # Find the index of the row with the matching ID in original DataFrame
            original_index = original[original.iloc[:, 7] == latest_id].index[0]
            original_row = original.iloc[original_index]

            # Extract and compare the entire rows
            latest_index = latest[latest.iloc[:, 7] == latest_id].index[0]
            latest_row = latest.iloc[latest_index]

            # Categorize based on comparison
            if original_row.equals(latest_row):
                original_not_updated_projs[latest_id] = original_row
            else:
                original_updated_projs[latest_id] = latest_row

        # enter here if id null or space or special characters in the ids excluding hyphen '-', id not in 87... or new-... or empty
        elif pd.isna(latest_id) or latest_id.isspace() or bool(re.search(r'[^a-zA-Z0-9\-]', latest_id)) or latest_id is '':
            if latest_titles[i] not in new_projs: # latest_titles[i] not in new_project
                # new_project as id: corresponding row including values of all columns
                new_projs[latest_titles[i]] = latest.iloc[i + 7]
            new_projs[latest_titles[i]] = latest.iloc[i + 7]
        else:
            # print id and say id error
            print(f'###### ID ERROR. Id: {latest_id} Title: {latest_titles[i]} ######')                       

    return new_projs, deleted_projs, original_not_updated_projs, original_updated_projs

In [96]:
# define column names
columns = ['SWG', 'Project Title', 'Status', 'On/Off/Treasury Budget', 'Humanitarian Aid', 'Project description', 
           'Project ID (in AMP)', 'Actual Approval Date', 'Actual Start Date', 'Actual Closure Date', 
           'Donor Group', 'Donor Agency', 'Implementing Agency', 'Type of Assistance', 'Mode of Payment', 
           'Aid Modality', 'Actual Commitments', 'Actual Disbursements', 'Undisbursed Balance']

# Load sheets from Excel files
sincesht = 'Existing projects approved sinc'
beforesht = 'Existing projects approved befo'
newsht = 'Record new projects'

deletedsht = 'Deleted projects'
updatedsht = 'Updated projects'

orig_file = 'ODA_Original_and_Latest/OrigHPN.xlsx'
lat_file = 'ODA_Original_and_Latest/LatHPN.xlsx'

original_sinc = pd.read_excel(orig_file, sheet_name=sincesht)
latest_sinc = pd.read_excel(lat_file, sheet_name=sincesht)

original_befo = pd.read_excel(orig_file, sheet_name=beforesht)
latest_befo = pd.read_excel(lat_file, sheet_name=beforesht)

# new_pr = pd.read_excel(lat_file, sheet_name=newsht)


# original_sinc = pd.read_excel('ODA_Original_and_Latest/OrigHPN.xlsx', sheet_name=sincesht)
# latest_sinc = pd.read_excel('ODA_Original_and_Latest/LatHPN.xlsx', sheet_name=sincesht)

# original_befo = pd.read_excel('ODA_Original_and_Latest/OrigHPN.xlsx', sheet_name=beforesht)
# latest_befo = pd.read_excel('ODA_Original_and_Latest/LatHPN.xlsx', sheet_name=beforesht)

# new_pr = pd.read_excel('ODA_Original_and_Latest/OrigHPN.xlsx', sheet_name=newsht)



for new project:
1. read it and add to new projects list resulting from since and before meaning there are new projects added to those sheets. if value is entered for more than 2 columns in a single row then consider it. 

In [97]:
# get comparison statistics
new_project_sinc, deleted_project_sinc, original_not_updated_projs_sinc, original_updated_projs_sinc = compare_stat(original_sinc, latest_sinc)
new_project_befo, deleted_project_befo, original_not_updated_projs_befo, original_updated_projs_befo = compare_stat(original_befo, latest_befo)

In [98]:


# write to excel
with pd.ExcelWriter('comparison.xlsx') as writer:
    pd.DataFrame.from_dict(new_project_sinc, orient='index').to_excel(writer, sheet_name='New Projs Since 2017')
    pd.DataFrame.from_dict(deleted_project_sinc, orient='index').to_excel(writer, sheet_name='Deleted Projs Since 2017')
    pd.DataFrame.from_dict(original_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Updated Projs Since 2017')
    pd.DataFrame.from_dict(original_not_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Not Updated Projs Since 2017')

    pd.DataFrame.from_dict(new_project_befo, orient='index').to_excel(writer, sheet_name='New Projs Before 2017')
    pd.DataFrame.from_dict(deleted_project_befo, orient='index').to_excel(writer, sheet_name='Deleted Projs Before 2017')
    pd.DataFrame.from_dict(original_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Updated Projs Before 2017')
    pd.DataFrame.from_dict(original_not_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Not Updated Projs Before 2017')
    

# read excel file back and change the column names for all sheets
df = pd.read_excel('comparison.xlsx', sheet_name=None)
sheets_to_exclude = []
for sheet in df:
    # remove the first two and the last 1 columns
    
    df[sheet] = df[sheet].iloc[:, 2:-1] # df[sheet] = df[sheet].iloc[:, 2:21]
    
    # Ensure 'columns' has the right length
    if len(columns) == len(df[sheet].columns):
        df[sheet].columns = columns
    else:
        sheets_to_exclude.append(sheet)

# write to excel
with pd.ExcelWriter('comparison.xlsx') as writer:
    workbook = writer.book
    for sheet in df:
        if sheet in sheets_to_exclude:
            print(f'###### Empty Sheet {sheet} has been excluded ######')
            continue
        df[sheet].to_excel(writer, sheet_name=sheet, index=False)
        # Access the workbook and the sheet
        
        worksheet = writer.sheets[sheet]

        # highlight the header cells
        # Define header format
        header_fill = PatternFill(start_color="D7E4BC", end_color="D7E4BC", fill_type="solid")
        thin_border = Border(left=Side(style='thin'), 
                             right=Side(style='thin'), 
                             top=Side(style='thin'), 
                             bottom=Side(style='thin'))
        header_font = Font(bold=True)
        header_alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)

        # Apply header format
        for cell in worksheet[1]:
            cell.fill = header_fill
            cell.border = thin_border
            cell.font = header_font
            cell.alignment = header_alignment

        # add filter to the header
        worksheet.auto_filter.ref = "A1:S1"

        # freeze the first row
        worksheet.freeze_panes = 'A2' 

        # Set column widths
        high = ['A', 'B']  
        medium = ['F', 'G', 'H', 'L', 'M', 'K',  'I', 'J', 'Q', 'R', 'S', 'T'] 
        low = ['C', 'D', 'E', 'N', 'O', 'P']

        for col in high:
            worksheet.column_dimensions[col].width = 30

        for col in medium:
            worksheet.column_dimensions[col].width = 18 

        for col in low:
            worksheet.column_dimensions[col].width = 12          

###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
